<a href="https://colab.research.google.com/github/EmilyCarroll-del/Predicting-Freezing-of-Gait-in-Parkinson-s-Disease-Using-ML/blob/main/Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/Colab Notebooks/'
%ls

/content/drive/MyDrive/Colab Notebooks
FFT.ipynb                                Test-Data/
RandomForestCorrect.ipynb                tlvmc-parkinsons-freezing-gait-prediction/
RandomForestFinalPreprocessedData.ipynb  Training-Data/
RandomForestNew.ipynb                    Untitled0.ipynb
ResNetFinal.ipynb.ipynb                  Untitled1.ipynb
ResNetNew.ipynb


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns


# Process Data

In [ ]:
#directory to each folder in train
train_defog_dir_Turn = '/content/drive/MyDrive/Colab Notebooks/Training-Data/defog_turn.csv'
train_defog_dir_SH= '/content/drive/MyDrive/Colab Notebooks/Training-Data/defog_start_hesitation.csv'
train_defog_dir_W= '/content/drive/MyDrive/Colab Notebooks/Training-Data/defog_walking.csv'

test_defog_dir_Turn= '/content/drive/MyDrive/Colab Notebooks/Test-Data/defog_turn_test.csv'
test_defog_dir_SH= '/content/drive/MyDrive/Colab Notebooks/Test-Data/defog_start_hesitation_test.csv'
test_defog_dir_W= '/content/drive/MyDrive/Colab Notebooks/Test-Data/defog_walking_test.csv'

In [ ]:
#directory to each folder in train
train_tdcsfog_dir_Turn = '/content/drive/MyDrive/Colab Notebooks/Training-Data/tdcsfog_turn.csv'
train_tdcsfog_dir_SH= '/content/drive/MyDrive/Colab Notebooks/Training-Data/tdcsfog_start_hesitation.csv'
train_tdcsfog_dir_W= '/content/drive/MyDrive/Colab Notebooks/Training-Data/tdcsfog_walking.csv'

test_tdcsfog_dir_Turn= '/content/drive/MyDrive/Colab Notebooks/Test-Data/tdcsfog_turn_test.csv'
test_tdcsfog_dir_SH= '/content/drive/MyDrive/Colab Notebooks/Test-Data/tdcsfog_start_hesitation_test.csv'
test_tdcsfog_dir_W= '/content/drive/MyDrive/Colab Notebooks/Test-Data/tdcsfog_walking_test.csv'

In [ ]:
# Process each defog subdirectory
defog_turn = pd.read_csv(train_defog_dir_Turn)
defog_SH = pd.read_csv(train_defog_dir_SH)
defog_W = pd.read_csv(train_defog_dir_W)

defog_turn_test = pd.read_csv(test_defog_dir_Turn)
defog_SH_test = pd.read_csv(test_defog_dir_SH)
defog_W_test = pd.read_csv(test_defog_dir_W)

In [ ]:
# Process each tdcsfog subdirectory
tdcsfog_turn = pd.read_csv(train_tdcsfog_dir_Turn)
tdcsfog_SH = pd.read_csv(train_tdcsfog_dir_SH)
tdcsfog_W = pd.read_csv(train_tdcsfog_dir_W)

tdcsfog_turn_test = pd.read_csv(test_tdcsfog_dir_Turn)
tdcsfog_SH_test = pd.read_csv(test_tdcsfog_dir_SH)
tdcsfog_W_test = pd.read_csv(test_tdcsfog_dir_W)

In [ ]:
tdcsfog_W_test.head()

,Unnamed: 0,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,Walking
0,0,-5.323772,-0.003925,-1.072463,2.504091,0.163969,0.973779,0.235721,-0.298644,0.581682,0
1,1,-5.506546,0.027250,-1.253309,0.937572,-1.036939,0.334648,0.488285,-0.432269,-0.113006,0
2,2,-4.989670,-0.825342,1.130794,-1.923538,-1.550589,-0.246075,0.087627,1.123411,0.648512,0
3,3,-2.409238,1.761802,4.071607,-2.063834,-0.411514,-0.254477,2.151966,-0.716654,0.334372,0
4,4,-1.547210,2.486314,2.351423,0.004478,-3.110257,-0.046188,1.411215,-0.831943,-0.160770,0


In [ ]:
#Drop the unnamed column
tdcsfog_turn.drop(columns= ["Unnamed: 0"])
tdcsfog_SH.drop(columns= ["Unnamed: 0"])
tdcsfog_W.drop(columns= ["Unnamed: 0"])

tdcsfog_turn_test.drop(columns= ["Unnamed: 0"])
tdcsfog_SH_test.drop(columns= ["Unnamed: 0"])
tdcsfog_W_test.drop(columns= ["Unnamed: 0"])

defog_turn.drop(columns= ["Unnamed: 0"])
defog_SH.drop(columns= ["Unnamed: 0"])
defog_W.drop(columns= ["Unnamed: 0"])

defog_turn_test.drop(columns= ["Unnamed: 0"])
defog_SH_test.drop(columns= ["Unnamed: 0"])
defog_W_test.drop(columns= ["Unnamed: 0"])



,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,Walking
0,4.317413,0.036561,-0.900469,-2.189015,0.545232,0.356858,0.436013,-0.013556,0
1,4.171481,-0.183856,-1.363225,1.152901,3.088963,-1.670770,0.855470,0.272183,0
2,4.122440,0.057086,-1.684743,2.061214,4.117958,-2.097520,1.165311,0.348128,0
3,4.053525,0.490108,-2.142500,3.149641,5.438940,-2.559314,1.597471,0.438389,0
4,4.455383,-1.215061,-0.306904,-0.120664,-0.061240,0.159117,0.027453,-0.036245,0
...,...,...,...,...,...,...,...,...,...
1280,2.035364,-0.481530,-0.204632,-0.410359,0.075901,-0.140757,0.410385,-0.482575,0
1281,1.674965,-1.073424,0.289501,0.104078,-0.414845,-0.202726,0.268935,-0.221085,0
1282,1.472851,-1.061828,0.452399,0.228601,-0.478237,-0.143147,0.255995,-0.263149,0
1283,1.723132,-1.295978,0.530420,0.050174,-0.338062,-0.255657,0.051483,0.020964,0


In [ ]:
#Defog Turn
X_train_defog_Turn = defog_turn.drop(columns = ["Turn"])
y_train_defog_Turn = defog_turn["Turn"]

X_test_defog_Turn =  defog_turn_test.drop(columns = ["Turn"])
y_test_defog_Turn = defog_turn_test["Turn"]

#Defog SH
X_train_defog_SH = defog_SH.drop(columns = ["StartHesitation"])
y_train_defog_SH = defog_SH["StartHesitation"]

X_test_defog_SH =  defog_SH_test.drop(columns = ["StartHesitation"])
y_test_defog_SH = defog_SH_test["StartHesitation"]

#Defog Walking
X_train_defog_W = defog_W.drop(columns = ["Walking"])
y_train_defog_W = defog_W["Walking"]

X_test_defog_W =  defog_W_test.drop(columns = ["Walking"])
y_test_defog_W = defog_W_test["Walking"]

In [ ]:
#Tdcsfog Turn
X_train_tdcsfog_Turn = tdcsfog_turn.drop(columns = ["Turn"])
y_train_tdcsfog_Turn = tdcsfog_turn["Turn"]

X_test_tdcsfog_Turn =  tdcsfog_turn_test.drop(columns = ["Turn"])
y_test_tdcsfog_Turn = tdcsfog_turn_test["Turn"]

#Tdcsfog SH
X_train_tdcsfog_SH = tdcsfog_SH.drop(columns = ["StartHesitation"])
y_train_tdcsfog_SH = tdcsfog_SH["StartHesitation"]

X_test_tdcsfog_SH =  tdcsfog_SH_test.drop(columns = ["StartHesitation"])
y_test_tdcsfog_SH = tdcsfog_SH_test["StartHesitation"]

#Tdcsfog Walking
X_train_tdcsfog_W = tdcsfog_W.drop(columns = ["Walking"])
y_train_tdcsfog_W = tdcsfog_W["Walking"]

X_test_tdcsfog_W =  tdcsfog_W_test.drop(columns = ["Walking"])
y_test_tdcsfog_W = tdcsfog_W_test["Walking"]

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
import scipy.stats as stats

# Model

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_auc_score

In [ ]:
# Fit the model
def gridSearchCV(X_train, y_train):
  param_grid = {
    'n_estimators': [70, 100, 150, 200],
    'max_depth': [None, 100,200,300],
    'min_samples_split': [2, 5, 7, 9],
    'min_samples_leaf': [1, 2, 4,7],
  }
  modelRF = RandomForestClassifier(random_state=42)

  # Define the GridSearchCV
  grid_search = GridSearchCV(estimator=modelRF, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1_macro')

  #Run the search
  grid_search.fit(X_train, y_train)

  # Get the best parameters and score
  print("Best parameters found: ", grid_search.best_params_)
  print("Best cross-validation score: ", grid_search.best_score_)
  return grid_search.best_estimator_


In [ ]:
bestMode = gridSearchCV(X_train_defog_Turn, y_train_defog_Turn)

Fitting 5 folds for each of 256 candidates, totalling 1280 fits


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score


In [ ]:
def runBestModel(bestMode, X_train, y_train, X_test, y_test):
  bestMode.fit(X_train, y_train)
  y_pred = bestMode.predict(X_test)
  print("Test Accuracy:", accuracy_score(y_test, y_pred))
  #print("Classification Report:\n", classification_report(y_test, y_pred))

  precision = precision_score(y_test, y_pred, average='weighted')
  recall = recall_score(y_test, y_pred, average='weighted')
  f1 = f1_score(y_test, y_pred, average='weighted')

  print(f"Precision: {precision}")
  print(f"Recall: {recall}")
  print(f"F1 Score: {f1}")

In [ ]:
runBestModel(bestMode,X_train_defog_Turn, y_train_defog_Turn, X_test_defog_Turn, y_test_defog_Turn )
print("Turn for Defog")

Test Accuracy: 0.6326848249027237
Precision: 0.40029008766219015
Recall: 0.6326848249027237
F1 Score: 0.49034581758428447
Turn for Defog


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
bestModeDefogSH = gridSearchCV(X_train_defog_SH, y_train_defog_SH)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 20}
Best cross-validation score:  0.8629460773461796


In [ ]:
print("Start Hesitation for Defog")
runBestModel(bestModeDefogSH,X_train_defog_SH, y_train_defog_SH, X_test_defog_SH, y_test_defog_SH )

Start Hesitation for Defog
Test Accuracy: 1.0
Precision: 1.0
Recall: 1.0
F1 Score: 1.0


In [ ]:
bestModeDefogW = gridSearchCV(X_train_defog_W, y_train_defog_W)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}
Best cross-validation score:  0.833365221545192


In [ ]:
print("Walking for Defog")
runBestModel(bestModeDefogW,X_train_defog_W, y_train_defog_W, X_test_defog_W, y_test_defog_W )

Walking for Defog
Test Accuracy: 0.977431906614786
Precision: 0.9553731320686157
Recall: 0.977431906614786
F1 Score: 0.9662766428242198


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
bestModeTdcsfogTurn = gridSearchCV(X_train_tdcsfog_Turn, y_train_tdcsfog_Turn)

Fitting 5 folds for each of 256 candidates, totalling 1280 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 70}
Best cross-validation score:  0.8932280837241908


In [ ]:
print("Turn for Tdcsfog")
runBestModel(bestModeTdcsfogTurn,X_train_tdcsfog_Turn, y_train_tdcsfog_Turn, X_test_tdcsfog_Turn, y_test_tdcsfog_Turn)

Turn for Tdcsfog
Test Accuracy: 0.37950138504155123
Precision: 0.14402130124845572
Recall: 0.37950138504155123
F1 Score: 0.2088019668702511


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
bestModeTdcsfogW = gridSearchCV(X_train_tdcsfog_W, y_train_tdcsfog_W)

NameError: name 'X_train_tdcsfog_W' is not defined

In [ ]:
print("Walking for Tdcsfog")
runBestModel(bestModeTdcsfogW,X_train_tdcsfog_W, y_train_tdcsfog_W, X_test_tdcsfog_W, y_test_tdcsfog_W)

Walking for Tdcsfog


NameError: name 'runBestModel' is not defined

In [ ]:
bestModeTdcsfogSH = gridSearchCV(X_train_tdcsfog_SH, y_train_tdcsfog_SH)

Fitting 3 folds for each of 81 candidates, totalling 243 fits
Best parameters found:  {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 20}
Best cross-validation score:  0.8348281731860943


In [ ]:
print("Start Hesitation for Tdcsfog")
runBestModel(bestModeTdcsfogSH,X_train_tdcsfog_SH, y_train_tdcsfog_SH, X_test_tdcsfog_SH, y_test_tdcsfog_SH)

Start Hesitation for Tdcsfog
Test Accuracy: 0.7423822714681441
Precision: 0.5511314369902012
Recall: 0.7423822714681441
F1 Score: 0.6326182790253183


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
